In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))
import pdb

In [ ]:
emb_hyp = {
    'audio/sample_rate': 8000,
    'embedder/n_fft': 512,
    'embedder/hop': 160,
    'embedder/n_mels': 40,
    'embedder/num_layers': 3,
    'embedder/lstm_hidden': 768,
    'embedder/emb_dim': 256,
    'embedder/pretrained_path': '/mnt/data/zhepei/outputs/tse_semi/spid/vfpt_spid/2021-08-29-22-19-31_blstm_softmax_8k_bs46_4sec/chkpt/chkpt_100000.pt',
    'embedder/trainable': False,
}

from spid_modules.embedder import Embedder
embedder = Embedder(emb_hyp)
embedder.eval()

In [ ]:
hyp = {
    'dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'valid_data': '/mnt/data/Speech/wsj_tse/mix_2_spk_cv/meta.csv',
    'base_folder_dm': '/mnt/data/wsj0.8k/si_tr_s/',
    'base_folder_dm_ext': 'wav',
    'sample_rate': 8000,
    'training_signal_len': 24000,
}

In [ ]:
from data.data_mixing import static_mixing_prep
import torch
import numpy as np
from speechbrain.dataio.batch import PaddedBatch

# Test static mixing 

In [ ]:
ds = static_mixing_prep(hyp, 'valid')
dl = torch.utils.data.DataLoader(
        ds,
        batch_size=hyp["dataloader_opts"]["batch_size"],
        num_workers=hyp["dataloader_opts"]["num_workers"],
        collate_fn=PaddedBatch,
        worker_init_fn=lambda x: np.random.seed(
            int.from_bytes(os.urandom(4), "little") + x
        ),
    )

In [ ]:
def listen_batch(batch):
    mix = batch['mix_sig'].data
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    diff = mix - s1 - s2
    print(abs(mix).max())
    print((diff**2).mean())
    sound(mix[0].numpy(), rate=hyp['sample_rate'], label='mix')
    sound(s1[0].numpy(), rate=hyp['sample_rate'], label='s1')
    sound(s2[0].numpy(), rate=hyp['sample_rate'], label='s2')
    sound(enr[0].numpy(), rate=hyp['sample_rate'], label='enr')

In [ ]:
def compute_cos_sim(v1, v2):
    return (v1 * v2).sum() / (torch.norm(v1) * torch.norm(v2))

def check_emb(batch):
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    s1_emb = embedder(s1)
    s2_emb = embedder(s2)
    enr_emb = embedder(enr)
    pos_sim = compute_cos_sim(s1_emb[0], enr_emb[0]).item()
    neg_sim = compute_cos_sim(s2_emb[0], enr_emb[0]).item()
    print('Positive sim: {} -- Negative sim: {} -- diff: {}'.format(pos_sim, neg_sim, pos_sim-neg_sim))

In [ ]:
for i, batch in enumerate(dl):  
    check_emb(batch)
    if i == 20:
        break

In [ ]:
for i, batch in enumerate(dl):  
    if i == 1:
        listen_batch(batch)
        break